In [1]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

In [16]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# Constants
alpha = 1

def calculate_heights(s, p_y, p_l_y, p_not_l_not_y):
    p_not_l_y = 1.0 - p_l_y
    p_not_y = 1.0 - p_y
    common_term = (p_l_y * p_y) + (p_not_l_not_y * p_not_y)

    heights = [
        alpha * (p_y ** 4) * (p_not_l_y ** 7),
        alpha * (p_y ** 3) * (p_not_l_y ** 6) * s * common_term,
        alpha * (p_y ** 2) * (p_not_l_y ** 5) * (s ** 2) * (common_term ** 2),
        alpha * (p_y) * (p_not_l_y ** 4) * (s ** 3) * (common_term ** 3),
        alpha * (p_not_l_y ** 3) * (s ** 4) * (common_term ** 4),
        alpha * (p_l_y) * (p_not_l_y ** 2) * (s ** 5) * (common_term ** 4),
        alpha * (p_l_y ** 2) * (p_not_l_y) * (s ** 6) * (common_term ** 4),
        alpha * (p_l_y ** 3) * (s ** 7) * (common_term ** 4)
    ]

    total = sum(heights)
    normalized_heights = [h / total for h in heights]
    
    return normalized_heights

initial_s = 1.0
initial_p_y = 0.5
initial_p_l_y = 0.5
initial_p_not_l_not_y = 0.5

initial_heights = calculate_heights(initial_s, initial_p_y, initial_p_l_y, initial_p_not_l_not_y)
df = pd.DataFrame({
    "State": [f"π{i+1}" for i in range(8)],
    "Height": initial_heights
})

fig = px.bar(df, x="State", y="Height", title="Interactive Stationary Distribution Chart")
fig.update_layout(
    yaxis=dict(range=[0, 1], tickformat=".2f"),
    shapes=[
        dict(type="line", x0=3.5, y0=0, x1=3.5, y1=1, line=dict(dash="dot", color="gray"))
    ],
    annotations=[
        dict(x=1.75, y=0.9, text="Forgotten", showarrow=False, font=dict(size=14, color="gray")),
        dict(x=5.5, y=0.9, text="Memorized", showarrow=False, font=dict(size=14, color="gray"))
    ]
)

# App layout
app.layout = html.Div(children=[
    html.H1(children="Distribution Chart"),
    dcc.Graph(id='distribution-graph', figure=fig),

    html.Div([
        html.Label("s"),
        dcc.Slider(id='s-slider', min=1.0, max=25.0, step=0.1, value=initial_s,
                   marks={i: str(i) for i in range(1, 26)}),

        html.Label("P(Y)"),
        dcc.Slider(id='p-y-slider', min=0, max=1, step=0.01, value=initial_p_y,
                   marks={i/10: str(i/10) for i in range(0, 11)}),

        html.Label("P(L|Y)"),
        dcc.Slider(id='p-l-y-slider', min=0, max=1, step=0.01, value=initial_p_l_y,
                   marks={i/10: str(i/10) for i in range(0, 11)}),

        html.Label("P(nL|nY)"),
        dcc.Slider(id='p-not-l-not-y-slider', min=0, max=1, step=0.01, value=initial_p_not_l_not_y,
                   marks={i/10: str(i/10) for i in range(0, 11)})
    ], style={'width': '80%', 'margin': 'auto'})
])

@app.callback(
    Output('distribution-graph', 'figure'),
    [
        Input('s-slider', 'value'),
        Input('p-y-slider', 'value'),
        Input('p-l-y-slider', 'value'),
        Input('p-not-l-not-y-slider', 'value')
    ]
)
def update_graph(s, p_y, p_l_y, p_not_l_not_y):
    heights = calculate_heights(s, p_y, p_l_y, p_not_l_not_y)
    updated_df = pd.DataFrame({
        "State": [f"{i+1}" for i in range(8)],
        " ": heights
    })
    
    fig = px.bar(updated_df, x="State", y=" ")
    fig.update_layout(
        yaxis=dict(range=[0, 1], tickformat=".2f"),
        shapes=[
            dict(type="line", x0=3.5, y0=0, x1=3.5, y1=1, line=dict(dash="dot", color="gray"))
        ],
        annotations=[
            dict(x=1.75, y=0.9, text="Forgotten", showarrow=False, font=dict(size=14, color="gray")),
            dict(x=5.5, y=0.9, text="Memorized", showarrow=False, font=dict(size=14, color="gray"))
        ]
    )
    
    return fig

if __name__ == '__main__':
    app.run(debug=True)
